read data

In [1]:
import time
import sys
import numpy as np
from sklearn.neighbors import KDTree
from pandas import read_csv
import numpy as np
from sklearn.neighbors import KDTree
import pandas as pd
from pandas import read_csv
from shapely.geometry import Polygon, Point, box
import rtree
import geopandas as gpd
import re
df = read_csv("sample-covid_contact_locations-infs4205.csv")
df

,id,longitude,latitude,date,time
0,1,151.25066,-23.840897,2021-12-28,00:01
1,2,153.01736,-27.466467,2021-12-27,00:00
2,3,151.25066,-23.840897,2021-12-27,12:00
3,4,146.81145,-19.267107,2021-12-27,12:30
4,5,145.77582,-16.921663,2021-12-27,00:00
...,...,...,...,...,...
2533,2534,152.97940,-27.409080,2021-03-13,16:00
2534,2535,152.98834,-27.398056,2021-03-12,15:30
2535,2536,152.98834,-27.398056,2021-03-11,15:30
2536,2537,152.98834,-27.398056,2021-03-10,14:30


task1

In [14]:
features=['longitude','latitude']

index_start = time.time()
# Write your code for creating indexes here...

df = read_csv("sample-covid_contact_locations-infs4205.csv")
kd_tree = KDTree(df[features],leaf_size=df[features].shape[0]+1)

#print('t0:', time.time() - index_start)

points = []

with open("task5_sample.txt") as file:
    for line in file:
        #Format and get point, k, and datetime
        patn = re.sub(r"[\([{})\]]", "", line)
        get_time = re.findall(r'"([^"]*)"', line)
        start_time=pd.to_datetime(get_time[0])
        print(start_time)
        patn = patn.split()
        d = float(patn[-1])/100
        #print("d =",d)
        patn = patn[0:-2]
        #print(patn)
        for i in range(0,len(patn),2):
            #print(i)
            points.append([patn[i], patn[i+1]])
        id = []
        #print(points)
        result,dist=kd_tree.query_radius(points, d, True)
        #print(result)
        #print(dist)
        for i in range(len(result)):
            for j in range(len(result[i])):
                #print(result[i][j])
                #print(pd.to_datetime(df.loc[result[i][j]]['date']))
                #print(start_time)
                if(pd.to_datetime(df.loc[result[i][j]]['date']) == start_time):
                    id.append(df.loc[result[i][j]]['id'])
                    #print(df.loc[result[i][j]]['id'])
        id.sort()
        id=list(dict.fromkeys(id))
        f = open("task5_results.txt", "a")
        for i in range(len(id)):
            print("~~~~~~~~",i," time~~~~~~~")
            f.write(str(id[i]))
            f.write("\n")
            print(id[i])
        f.close()
            



2021-07-30 00:00:00
~~~~~~~~ 0  time~~~~~~~
1393
~~~~~~~~ 1  time~~~~~~~
1394
~~~~~~~~ 2  time~~~~~~~
1395
~~~~~~~~ 3  time~~~~~~~
1399
~~~~~~~~ 4  time~~~~~~~
1412
~~~~~~~~ 5  time~~~~~~~
1427
~~~~~~~~ 6  time~~~~~~~
1428
~~~~~~~~ 7  time~~~~~~~
1430
~~~~~~~~ 8  time~~~~~~~
1434
~~~~~~~~ 9  time~~~~~~~
1436
~~~~~~~~ 10  time~~~~~~~
1437
2021-06-30 00:00:00
~~~~~~~~ 0  time~~~~~~~
2117
~~~~~~~~ 1  time~~~~~~~
2117
~~~~~~~~ 2  time~~~~~~~
2118
~~~~~~~~ 3  time~~~~~~~
2118
~~~~~~~~ 4  time~~~~~~~
2120
~~~~~~~~ 5  time~~~~~~~
2122
~~~~~~~~ 6  time~~~~~~~
2122
~~~~~~~~ 7  time~~~~~~~
2122
~~~~~~~~ 8  time~~~~~~~
2122
~~~~~~~~ 9  time~~~~~~~
2123
~~~~~~~~ 10  time~~~~~~~
2123
~~~~~~~~ 11  time~~~~~~~
2123
~~~~~~~~ 12  time~~~~~~~
2123


task2

In [3]:
#prepare dataset
features=['longitude','latitude']
df = read_csv("sample-covid_contact_locations-infs4205.csv")
with open("task2_sample.txt") as file:
    for line in file:
        patn = re.sub(r"[\([{})\]]", "", line)
        time=re.findall(r'"([^"]*)"', line)
        start_time=pd.to_datetime(time[0])
        #print(start_time)
        end_time=pd.to_datetime(time[1])
        #print(end_time)
        patn = patn.split()
        point=[[float(patn[0]),float(patn[1])]]
        #print(point)
        K=int(patn[2])
        #print(K)
        filtered_dataset = df
        filtered_dataset['date_time'] = pd.to_datetime(df['date'] + ' ' + df['time'])
        filtered_dataset = filtered_dataset[(filtered_dataset['date_time'] >= start_time) & (filtered_dataset['date_time'] <= end_time)]
        
        filtered_dataset.reset_index(inplace=True, drop=True)
        
        kd_tree=KDTree(filtered_dataset[features],leaf_size=filtered_dataset[features].shape[0]+1)


        distances, ndx = kd_tree.query(point, K, return_distance=True)
        result=filtered_dataset.loc[ndx[0]]
        result['dist']=distances[0,:]
        result=result.sort_values(['dist','id'],ascending=[True,True])
        print(result['id'])


47     2273
48     2274
103    2329
104    2330
76     2302
49     2275
50     2276
92     2318
Name: id, dtype: int64


task3

In [4]:
from shapely.geometry import Polygon, Point, box
import rtree
import geopandas as gpd
import time
rectangle=[]

index_start = time.time()
# Write your code for the qindex here...
idx = rtree.index.Index()

with open("task3_sample.txt") as file:
    for line in file:
        patn = re.sub(r"[\([{})\]]", "", line)
        x = patn.split()
        rectangle.append(box(float(x[0]), float(x[1]), float(x[2]),float(x[3])))

for pos, poly in enumerate(rectangle):
    idx.insert(pos, poly.bounds)

data = pd.read_csv("sample-covid_contact_locations-infs4205.csv")
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.longitude, data.latitude))



print('t0:', time.time() - index_start)


points = [(x,y) for x,y in zip(gdf['geometry'].x , gdf['geometry'].y)]
#points


query_start = time.time()
f = open("test3.txt", "a")
# Write your code for the querying here...
for k in enumerate(rectangle):
    for i,pt in enumerate(points):
        #print(pt)
        point=Point(pt)
        if(idx.intersection(point.bounds)):
            if(point.intersects(rectangle[k[0]])):
                result = gdf.loc[i]
                f.write(str(result['id']))
                f.write("\n")
                #print(result['id'])

f.close()
print('t1:', time.time() - query_start)    

t0: 0.0606689453125
t1: 0.08199310302734375


/Users/xingyuhe/opt/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:35: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  out = from_shapely(data)


task4

In [5]:

import re
import time

rectangle=[]
start_time=[]
end_time=[]


index_start = time.time()
# Write your code for the qindex here...
idx = rtree.index.Index()

with open("task4_sample.txt") as file:
    for line in file:
        patn = re.sub(r"[\([{})\]]", "", line)
        get_time=re.findall(r'"([^"]*)"', line)
        start_time.append(pd.to_datetime(get_time[0]))
        #print(start_time)
        end_time.append(pd.to_datetime(get_time[1]))
        #print(end_time)
        x = patn.split()
        rectangle.append(box(float(x[0]), float(x[1]), float(x[2]),float(x[3])))

#print(start_time, end_time)

for pos, poly in enumerate(rectangle):
    idx.insert(pos, poly.bounds)

data = pd.read_csv("sample-covid_contact_locations-infs4205.csv")
data['date_time'] = pd.to_datetime(data['date'] + ' ' + data['time'])
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.longitude, data.latitude))



print('t0:', time.time() - index_start)


points = [(x,y) for x,y in zip(gdf['geometry'].x , gdf['geometry'].y)]

query_start = time.time()
f = open("test4.txt", "a")
# Write your code for the querying here...
for k in enumerate(rectangle):
    for i,pt in enumerate(points):
        #print(pt)
        point=Point(pt)
        if(idx.intersection(point.bounds)):
            if(point.intersects(rectangle[k[0]])):
                result = gdf.loc[i]
                if(result['date_time']>=start_time[k[0]] and result['date_time']<=end_time[k[0]]):
                    #print("~~~ok")
                    f.write(str(result['id']))
                    f.write("\n")
                    #print('\n================================')

f.close()
print('t1:', time.time() - query_start)   

t0: 0.06812787055969238
t1: 0.08715319633483887
